In [ ]:
import cv2
import numpy as np
cap = cv2.VideoCapture("tom.mkv")
while True:
    ret,frame=cap.read()
    frame_height=frame.shape[0]
    frame_width=frame.shape[1]    
    frame_blob=cv2.dnn.blobFromImage(frame, 1/255, (416,416), swapRB=True, crop=False)
    
    labels=["person","bicycle","car","motorcycle","airplane","bus","train","truck","boat","trafficlight","firehydrant","stopsign","parkingmeter","bench","bird","cat","dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack","umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sportsball","kite","baseballbat","baseballglove","skateboard","surfboard","tennisracket","bottle","wineglass","cup","fork","knife","spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hotdog","pizza","donut","cake","chair","sofa","pottedplant","bed","diningtable","toilet","tvmonitor","laptop","mouse","remote","keyboard","cellphone","microwave","oven","toaster","sink","refrigerator","book","clock","vase","scissors","teddybear","hairdrier","toothbrush"]
    
    colors = ["100,150,250","60,15,20","10,110,200","160,130,150","80,110,120"]
    # convert them to integer
    colors = [np.array(color.split(",")).astype("int") for color in colors]
    colors=np.array(colors)
    colors=np.tile(colors,(18,1))
    
    model=cv2.dnn.readNetFromDarknet("yolov3.cfg","yolov3.weights")
    layers=model.getLayerNames()
    output_layers = [layers[layer[0]-1] for layer in model.getUnconnectedOutLayers()]
    
    model.setInput(frame_blob)
    detection_layers = model.forward(output_layers)
    
    
    #### NON MAXIMUM SUPPRESSION OPERATION NEEDED!
    ids_list = []
    boxes_list = []
    confidences_list = []
    
    
    
    for detection_layer in detection_layers:
        for object_detection in detection_layer:

            scores = object_detection[5:]
            #first 5 for bounding
            #the highest score will be predicted
            predicted_id=np.argmax(scores)
            #np.argmax returns you the index of the maximum value in the array.
            confidence = scores[predicted_id]  

            if confidence > 0.4:
                # you can draw the bounding box, confidence high enough
                # but first, you need to find which label is that to write 
                label = labels[predicted_id]
                bounding_box = object_detection[0:4] * np.array([frame_width,frame_height,frame_width,frame_height]) #make the coordinates
                (box_center_x,box_center_y,box_width,box_height)=bounding_box.astype("int") # float>int
                
                start_x=int(box_center_x-(box_width/2))
                start_y=int(box_center_y-(box_height/2))

                # ------- Non Max Suppression
                ids_list.append(predicted_id)
                confidences_list.append(float(confidence))
                boxes_list.append([start_x,start_y,int(box_height),int(box_width)])
                #--------

                # ------
                max_ids = cv2.dnn.NMSBoxes(boxes_list,confidences_list,0.5,0.4) #optimal values, you can change, 0.4=threshold 
                for max_id in max_ids:
                    max_class_id = max_id[0]
                    box = boxes_list[max_class_id]
                    start_x = box[0]
                    start_y = box[1]
                    box_width= box[2]
                    box_height= box[3]
                    predicted_id = ids_list[max_class_id]
                    labels=labels[predicted_id]
                    confidence = confidences_list[max_class_id]
                    # ------

                    end_x=start_x+box_width
                    end_y=start_y+box_height

                    box_color = colors[predicted_id]
                    #for each object we can have a different color this way
                    box_color = [int(each) for each in box_color]

                    cv2.rectangle(frame,(start_x,start_y),(end_x,end_y),box_color,1)
                    cv2.putText(frame,label,(start_x,start_y-13),cv2.FONT_HERSHEY_COMPLEX,0.5,box_color,1)
                    
                    #%%       
        cv2.imshow("Detect!",frame) 
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
cap.release()
cv2.destroyAllWindows()
    

### References:
* https://pjreddie.com/darknet/yolo/
* udemy:oguzhangurbuz
